### ERDDAP 2018 Oculus Glider QuickLooks and Analysis

#### Puget Sound Spring Test for SG403

Glider Data is subsetted into various components to be hosted on ERDDAP due to mulitple time dimensions


In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import datetime

server_url = 'http://downdraft.pmel.noaa.gov:8080/erddap'

e = ERDDAP(server=server_url)

In [2]:
df = pd.read_csv(e.get_search_url(response='csv', search_for='sg403'))

The dataset is spread across multiple datasets in the erddap server so we need to join them all together

In [3]:
sg403 = df['Dataset ID'].values
print(sg403)

['sg403_PS_spring18' 'sg403_PS_spring18_aanderaa'
 'sg403_PS_spring18_wetlabs']


In [4]:
variables = [e.get_var_by_attr(dataset_id=sg, standard_name=lambda v: v is not None) for sg in sg403]
print(variables)
variables = [e.get_var_by_attr(dataset_id=sg, standard_name=lambda v: v is None) for sg in sg403]
print(variables)

[['time', 'density', 'ctd_time', 'ctd_depth', 'latitude', 'longitude', 'salinity', 'temperature', 'eng_depth'], ['aa4330_temp', 'time', 'aanderaa4330_dissolved_oxygen', 'aa4330_airsat'], ['time']]
[['profileid', 'NC_GLOBAL', 'pressure'], ['profileid_aand', 'NC_GLOBAL'], ['wlbb2fl_sig700nm_adjusted', 'wlbb2fl_temp', 'wlbb2fl_sig470nm_adjusted', 'wlbb2fl_sig695nm_adjusted', 'NC_GLOBAL', 'profileid_wetlabs']]


In [18]:
constraints = {
    'time>=': '2018-01-01T00:00:00Z',
    'time<=': str(datetime.datetime.today()),
}

variables = {'sg403_PS_spring18':['latitude', 
             'ctd_depth', 
             'longitude', 
             'ctd_time', 
             'eng_depth', 
             'salinity',
             'density', 
             'time', 
             'temperature']}
"""
variables = {'sg403_PS_spring18_wetlabs':['wlbb2fl_sig695nm_adjusted',
             'wlbb2fl_sig470nm_adjusted', 
             'wlbb2fl_sig700nm_adjusted', 
             'wlbb2fl_temp',
             'profileid_wetlabs']}

variables = {'sg403_PS_spring18_aanderaa':['aanderaa4330_dissolved_oxygen',
             'aa4330_airsat',
             'aa4330_temp']}"""



"\nvariables = {'sg403_PS_spring18_wetlabs':['wlbb2fl_sig695nm_adjusted',\n             'wlbb2fl_sig470nm_adjusted', \n             'wlbb2fl_sig700nm_adjusted', \n             'wlbb2fl_temp',\n             'profileid_wetlabs']}\n\nvariables = {'sg403_PS_spring18_aanderaa':['aanderaa4330_dissolved_oxygen',\n             'aa4330_airsat',\n             'aa4330_temp']}"

In [19]:
d = ERDDAP(server=server_url,
           protocol='tabledap',
           response='csv'
          )
d.dataset_id='sg403_PS_spring18'
d.variables=variables['sg403_PS_spring18']
d.constraints=constraints

download_url = d.get_download_url()

In [20]:
import requests


def humansize(nbytes):
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    k = 0
    while nbytes >= 1024 and k < len(suffixes)-1:
        nbytes /= 1024.
        k += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[k])

r = requests.head(download_url)
nbytes = float(r.headers['Content-Length'])
humansize(nbytes)

'133.45 MB'

In [ ]:


from requests.exceptions import HTTPError

dfs = {}
for index,row in df.iterrows():
    if row['Dataset ID'] in sg403:
        print(row['Dataset ID'])
        try:
            e = ERDDAP(server=server_url,
                protocol='tabledap',
                response='csv',
            )
            e.dataset_id=row['Dataset ID']
            e.constraints=constraints
            e.variables=variables[row['Dataset ID']]
        except HTTPError:
            print('Failed to generate url {}'.format(row['Dataset ID']))
            continue
        dfs.update({row['Dataset ID']: e.to_pandas(
                                index_col='time',
                                parse_dates=True,
                                skiprows=(1,)  # units information can be dropped.
                                )})
    